# Decorators

My notes from 'Decorators and descriptors decoded - PyCon 2017' (https://www.youtube.com/watch?v=81S01c9zytE)

## Functions as first-class objects

Python functions can be:
+ created at runtime
+ assigned to a variable or element in data structure
+ passed as an argument to a function
+ returned as the result of a function

In [1]:
fruits = ["banana", "grapefruit", "lime", "pineapple"]
sorted(fruits, key=len)

['lime', 'banana', 'pineapple', 'grapefruit']

In [2]:
def fibonacci(n:int) -> int:
    """returns the nth Fibonacci number"""
    a, b = 0, 1
    while n > 0:
        a, b = b, a + b
        n -= 1
    return a

In [3]:
fibonacci.__doc__

'returns the nth Fibonacci number'

In [4]:
fibonacci.__annotations__

{'n': int, 'return': int}

In [5]:
fibonacci.__code__.co_varnames # names of local variables

('n', 'a', 'b')

In [6]:
fibonacci

<function __main__.fibonacci>

In [7]:
from inspect import signature
signature(fibonacci).parameters

mappingproxy({'n': <Parameter "n:int">})

In [8]:
def deco(f):
    def inner():
        return "inner result"
    return inner

@deco
def target():
    return "target result"

In [9]:
target()

'inner result'

In [10]:
target

<function __main__.deco.<locals>.inner>

## -----------------

In [3]:
def f1(a):
    print(a)
    print(some_undefined_variable_1)


This raises error:

#### ---------------------
This doesn't rase an error:

In [8]:
some_variable_2 = 4

def f2(a):
    print(a)
    print(some_variable_2)
f2(5)

5
4


#### -----------

In [11]:
some_variable_instance = 5

def f3(a):
    print(a)
    print(some_variable_instance)
    some_variable_instance = 8


This will get an error when called:

Declarative code - not executing user code, just tells interpreter something. np. 'global x'

# -----------------------------

In [20]:
class Averager():
    
    def __init__(self):
        self.series = []
        
    def __call__(self, value):
        self.series.append(value)
        return sum(self.series)/len(self.series)

In [12]:
def averager_maker():
    series = []
    def averager(argument_to_add_to_series):
        series.append(argument_to_add_to_series)
        return sum(series)/len(series)
    return averager

avg = averager_maker()
avg(10)

10.0

In [13]:
avg(11)

10.5

In [14]:
avg(12)

11.0

In [21]:
avg.__code__.co_varnames

('argument_to_add_to_series',)

In [22]:
avg.__code__.co_freevars

('series',)

In [23]:
avg.__closure__

(<cell at 0x00000195B3B84E58: list object at 0x00000195B3C79448>,)

In [26]:
avg.__code__

<code object averager at 0x00000195B3C661E0, file "<ipython-input-12-eecf9b0dec65>", line 3>

In [27]:
dir(avg.__code__)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'co_argcount',
 'co_cellvars',
 'co_code',
 'co_consts',
 'co_filename',
 'co_firstlineno',
 'co_flags',
 'co_freevars',
 'co_kwonlyargcount',
 'co_lnotab',
 'co_name',
 'co_names',
 'co_nlocals',
 'co_stacksize',
 'co_varnames']

In [34]:
avg.__closure__

(<cell at 0x00000195B3B84E58: list object at 0x00000195B3C79448>,)

In [37]:
cell = avg.__closure__[0]

In [39]:
cell.cell_contents

[10, 11, 12]

## --------------------------------

In [42]:
def make_averager_2():
    count = 0
    total = 0
    
    def averager(value):
        nonlocal count, total  # bez tego nie zadziała, bo 'count += 1' to definicja zmiennej 'count = count + 1'
                               # a definicja zmiennej w namespace sprawia, że python nie szuka jej w szerszym namespace
        count += 1
        total += value
        return total / count
    
    return averager

In [44]:
def make_averager_3():  # tak można to obejść w 2.7, bo tam nie ma 'nonlocal'
    d = {"count": 0, "total": 0}
    
    def averager(value):
        d["count"] += 1
        d["total"] += value
        return d["total"] / d["count"]
    
    return averager
av_1 = make_averager_3()
av_1(10)

10.0

In [46]:
av_1(12)

11.333333333333334

## Decorators

In [60]:
def floatify(f):
    def floated(n):
        rv = float(f(n))
        return rv
    return floated

In [61]:
def square(n):
    return n * n
square(3)

9

In [62]:
@floatify
def square(n):
    return n * n
square(3)

9.0

#### ---------------

In [63]:
square

<function __main__.floatify.<locals>.floated>

In [66]:
from functools import wraps

In [68]:
def floatify(f):
    
    @wraps(f)
    def floated(n):
        rv = float(f(n))
        return rv
    
    return floated

@floatify
def square(n):
    return n*n
square

<function __main__.square>

In [69]:
# <function __main__.square> zamiast <function __main__.floatify.<locals>.floated>

## Parametrized Decorators

decoratror factories

In [16]:
def decorator_factory(parameter):
    def decorator(func):
        def inner(a, b):
            print(parameter)
            rv = func(a, b)
            return rv
        return inner
    return decorator

@decorator_factory("this will be printed")  # function_to_decorate = decorator_factory(parameter)(function_to_decorate)
def function_to_decorate(a, b):
    return (a + b)

function_to_decorate


<function __main__.decorator_factory.<locals>.decorator.<locals>.inner>

In [18]:
def f_to_decorate(a, b):
    return (a + b)
f_to_decorate = decorator_factory(10)(f_to_decorate)
f_to_decorate

<function __main__.decorator_factory.<locals>.decorator.<locals>.inner>

## Memorizing Decorator

In [22]:
calls = []
def fibonacci(n):
    calls.append(1)
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)
print(fibonacci(7), sum(calls))

13 41


In [24]:
from functools import lru_cache  # lru: list recently used
calls = []
@lru_cache()
def fibonacci(n):
    calls.append(1)
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)
print(fibonacci(7), sum(calls))

13 8


## Class based decorators

In [36]:
class Deco(object):
    def __call__(self, func):
        def inner(a, b):
            print("Decorated")
            rv = func(a, b)
            return rv
        return inner
    
@Deco()  # function = Deco()(function)
def function(a, b):
    return a + b

function(1, 2)

Decorated


3

Don't make decorators from scratch:
- at minimum, use @functools.wraps
- use Graham Dumpleton's <b>wrapt</b> or Michele Simionato's <b>decorator</b> package

# Descriptors

In [44]:
class Dog:
    paws = 4
fido = Dog()
fido.paws

4

In [45]:
fido.paws = 3

In [46]:
fido.paws

3

In [47]:
Dog.paws

4

In [ ]:
# ---------------------------------------

In [48]:
class Customer:
    
    def __init__(self, name, email, fidelity=0):
        self.name = name
        self.email = email
        self.fidelity = fidelity
        
    def full_email(self):
        return '{0} <{1}>'.format(self.name, self.email)

#### --------------------

In [50]:
class Square1000:
    def __init__(self):
        self.value = 0
    def __get__(self):
        return self.value + 1000
    def __set__(self, value):
        self.value = value**2
        
sq = Square1000()
sq